In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_derieden,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

# 1. 基金分类
-----------------------

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
today = dt.date(2015, 1, 30)
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标混合型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(report_dates_begin, 2)

In [4]:
# 获取相关组合情况

basic_info = fd_basicinfo(security_ids, current_date)
type_info = fd_typeclass(security_ids, current_date)
hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

In [5]:
df = pd.merge(basic_info, type_info, on="SECURITYID")
df = pd.merge(df, asset_port, on="SECURITYID", how="left")
df = pd.merge(df, hkport_info, on="SECURITYID", how="left")
if "ACCNETMKTCAP" in df:
    df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
else:
    df["ACCNETMKTCAP"] = 0.0

In [6]:
# 分类
flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
df.loc[flag, "混合型（子类）"] = "港股通混合型基金"
df.loc[(~flag) & (df["EQUITYINVERTO"] >= 70), "混合型（子类）"] = "高权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"

In [7]:
df

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP,混合型（子类）
0,1030000001,华夏复兴混合型证券投资基金,None,000031,证券投资基金,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),85.680000,0.0,高权益仓位混合型基金
1,1030000001,华夏复兴混合型证券投资基金,None,000031,证券投资基金,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.3,偏股型基金（股票上下限60%-95%）,85.680000,0.0,高权益仓位混合型基金
2,1030000006,华夏成长证券投资基金,None,000001,证券投资基金,成长型,2,混合基金,2.1,偏股型基金,2.1.2,偏股型基金(股票上限80%),65.283333,0.0,中高权益仓位混合型基金
3,1030000007,华夏大盘精选证券投资基金,华夏大盘精选混合,000011,证券投资基金,增值型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),82.620833,0.0,高权益仓位混合型基金
4,1030000008,华夏优势增长混合型证券投资基金,None,000021,证券投资基金,稳健成长型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),79.129167,0.0,高权益仓位混合型基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,1030000112,国投瑞银融华债券型证券投资基金,None,121001,证券投资基金,收益型,2,混合基金,2.4,偏债型基金,2.4.1,偏债型基金,38.870833,0.0,中权益仓位混合型基金
256,1030001114,银河强化收益债券型证券投资基金,None,519676,证券投资基金,稳健成长型,3,债券基金,3.2,普通债券型基金,3.2.3,普通债券型基金(二级A类),10.041667,0.0,低权益仓位混合型基金
257,1030000124,银河收益证券投资基金,None,151002,证券投资基金,稳健成长型,2,混合基金,2.4,偏债型基金,2.4.1,偏债型基金,5.093333,0.0,低权益仓位混合型基金
258,1030000180,南方宝元债券型基金,南方宝元债券,202101,证券投资基金,稳健成长型,2,混合基金,2.4,偏债型基金,2.4.1,偏债型基金,30.825000,0.0,中权益仓位混合型基金


## 1.1 Put it all totether

In [8]:
def create_mutua_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 2)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
    hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, asset_port, on="SECURITYID", how="left")
    df = pd.merge(df, hkport_info, on="SECURITYID", how="left").drop_duplicates()
    if "ACCNETMKTCAP" in df:
        df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
    else:
        df["ACCNETMKTCAP"] = 0.0
    
    flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
    df.loc[flag, "混合型（子类）"] = "港股通混合型基金"
    df.loc[(~flag) & (df["EQUITYINVERTO"] >= 70), "混合型（子类）"] = "高权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"
    
    return df

In [9]:
%%time

create_mutua_fund_info("20211111").groupby( "混合型（子类）").count()

Wall time: 4.83 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP
混合型（子类）,,,,,,,,,,,,,,
中权益仓位混合型基金,143,143,103,143,143,143,143,143,143,143,143,143,143,143
中高权益仓位混合型基金,196,196,103,196,196,196,196,196,196,196,196,196,196,196
低权益仓位混合型基金,507,507,425,507,507,507,507,507,507,507,507,507,507,507
港股通混合型基金,17,17,6,17,17,17,17,17,17,17,17,17,17,17
高权益仓位混合型基金,1190,1190,411,1190,1190,1190,1190,1190,1190,1190,1190,1190,1190,1190


# 2. 基金指数
-----------------

In [10]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 TQ_FD_DERIVEDN 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "REPAIRUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [11]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_mutua_fund_info(trade_dt)
    pre_nav_info = fd_derieden(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_derieden(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("混合型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [12]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="混合型（子类）", values="chg.").to_excel("020_混合型基金_bak.xlsx")

In [13]:
final_report.pivot_table(index="level_0", columns="混合型（子类）", values="chg.")

混合型（子类）,中权益仓位混合型基金,中高权益仓位混合型基金,低权益仓位混合型基金,港股通混合型基金,高权益仓位混合型基金
level_0,,,,,
20150430,0.158706,0.340919,0.104259,NaN,0.404668
20150731,-0.035837,-0.068951,0.003793,NaN,-0.119720
20151030,-0.000123,0.007219,0.001074,NaN,-0.010361
20160129,-0.069494,-0.116434,-0.016718,NaN,-0.161958
20160429,0.049689,0.073769,0.013842,NaN,0.084470
20160729,0.028211,0.033609,0.016797,NaN,0.037625
20161031,0.026168,0.022514,0.012184,NaN,0.025027
20170126,-0.027615,-0.026839,-0.007877,NaN,-0.041254
20170428,0.008647,0.033472,0.007420,NaN,0.033349
